# Shuffle playlist
This notebook contains a Python script to shuffle the custom order of your Spotify playlist. By default, Spotify maintains a consistent ordering of tracks in a playlist, which can result in a predictable listening experience. This notebook solves this experience by providing a quick solution that reorders your tracks randomly.


You'll need your own `CLIENT_ID`, `CLIENT_SECRET`, `USER_ID` and `REDIRECT_URI` credentials to proceed.

In [1]:
import os
import pandas as pd
import re
import requests
import time
import json
from utils import *
%load_ext dotenv
%dotenv

In [2]:
# Retrieve credentials from the environment variables
CLIENT_ID = os.environ['CLIENT_ID']
CLIENT_SECRET = os.environ['CLIENT_SECRET']
USER_ID = os.environ['USER_ID']
REDIRECT_URI = os.environ['REDIRECT_URI']
SCOPE = 'playlist-modify-public playlist-modify-private'

# Get authorization
Initiates the authorization flow for Spotify API using port 8889. It returns the callback URL to `callback_url` variable after the user grants access.

The `CallbackHandler` class is designed to handle incoming GET requests to extract the callback URL.

In [3]:
import http.server
import socketserver
import webbrowser

In [4]:
class CallbackHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, format, *args):
        """
        Override log_message method to suppress server logs.
        """
        pass
    
    def do_GET(self):
        """
        Handle the GET request to extract the callback URL.
        """

        # Declare callback_url as global to modify the global variable
        global callback_url

        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

        # Extract the callback URL from self.path
        callback_url = self.path

In [5]:
# Spotify authorization URL
auth_url = 'https://accounts.spotify.com/authorize'
callback_url = None

# Parameters for the authorization request
params = {
    'client_id': CLIENT_ID,
    'response_type': 'code',
    'redirect_uri': REDIRECT_URI,
    'scope': SCOPE
}

# Send the authorization request and get the response URL
auth_response = requests.get(auth_url, params=params)
auth_response_url = auth_response.url

# Start the local server to handle the callback
with socketserver.TCPServer(('localhost', 8889), CallbackHandler) as httpd:
    # Open the authorization URL in a browser
    webbrowser.open(auth_response_url)

    # Handle a single request
    httpd.handle_request()

# Generate access token

In [6]:
AUTH_CODE = callback_url[15:]

In [7]:
access_token = request_an_access_token(CLIENT_ID, CLIENT_SECRET, AUTH_CODE, REDIRECT_URI)

# Read data
Remember to run `get_spotify_data.ipynb` first to have the most updated list of tracks in a playlist.

In [8]:
df_playlist = pd.read_csv('./data/playlists.csv')
df_playlist.head()

,id,name,tracks.total
0,13Qsm0axSPpL11U5yGhwFS,Awkward,2
1,730Ce3gbzbasRtac5l8eXs,My Songs,1076
2,2gibcQ6TCJyyQgdJaDNWsT,Nicole&Shaun Wedding Playlist :),106
3,78TQufEn9zE564Is7DKk46,Karaoke,10
4,6mAJ1EqzlLuD5o97BB1VNP,1) 29.10 Pre-walk in,16


In [9]:
df_tracks = pd.read_csv('./data/tracks.csv')
df_tracks.head()

,href,track.id,track.name,playlist_id
0,https://api.spotify.com/v1/playlists/13Qsm0axS...,4CPYZtb4tX2V03jcsJAZCD,Where's Kevin (From 'Overcooked! 2'),13Qsm0axSPpL11U5yGhwFS
1,https://api.spotify.com/v1/playlists/13Qsm0axS...,4cmRCH5q4Mp5DKqsGkQ2eu,"Super Mario Theme (From ""Super Mario"")",13Qsm0axSPpL11U5yGhwFS
2,https://api.spotify.com/v1/playlists/730Ce3gbz...,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,730Ce3gbzbasRtac5l8eXs
3,https://api.spotify.com/v1/playlists/730Ce3gbz...,4jPy3l0RUwlUI9T5XHBW2m,Mood (feat. iann dior),730Ce3gbzbasRtac5l8eXs
4,https://api.spotify.com/v1/playlists/730Ce3gbz...,2tGvwE8GcFKwNdAXMnlbfl,happier,730Ce3gbzbasRtac5l8eXs


# Select playlist to update

In [10]:
playlist_id = '730Ce3gbzbasRtac5l8eXs'

df_playlist[df_playlist['id'] == playlist_id]

,id,name,tracks.total
1,730Ce3gbzbasRtac5l8eXs,My Songs,1076


# Remove tracks from playlist
Do this before adding shuffled tracks to avoid duplicated tracks in your playlist.

In [11]:
df_delete = df_tracks[df_tracks['playlist_id'] == playlist_id].reset_index(drop = True)
df_delete['group'] = (df_delete.index // 100) + 1
df_delete.head()

,href,track.id,track.name,playlist_id,group
0,https://api.spotify.com/v1/playlists/730Ce3gbz...,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,730Ce3gbzbasRtac5l8eXs,1
1,https://api.spotify.com/v1/playlists/730Ce3gbz...,4jPy3l0RUwlUI9T5XHBW2m,Mood (feat. iann dior),730Ce3gbzbasRtac5l8eXs,1
2,https://api.spotify.com/v1/playlists/730Ce3gbz...,2tGvwE8GcFKwNdAXMnlbfl,happier,730Ce3gbzbasRtac5l8eXs,1
3,https://api.spotify.com/v1/playlists/730Ce3gbz...,7qEHsqek33rTcFNT9PFqLf,Someone You Loved,730Ce3gbzbasRtac5l8eXs,1
4,https://api.spotify.com/v1/playlists/730Ce3gbz...,0u2P5u6lvoDfwTYjAADbn4,lovely (with Khalid),730Ce3gbzbasRtac5l8eXs,1


In [12]:
for i in df_delete['group'].unique():
    df_tmp = df_delete[df_delete['group'] == i]
    tracks = df_tmp['track.id'].apply(lambda x: {'uri': 'spotify:track:' + x}).tolist()
    remove_tracks_playlist(playlist_id, tracks, access_token)
    print('Done group ' + str(i))

Done group 1
Done group 2
Done group 3
Done group 4
Done group 5
Done group 6
Done group 7
Done group 8
Done group 9
Done group 10
Done group 11


# Add tracks to playlist
Add the tracks back to your playlist based on a random shuffle using `random_state = 42`.

To create a new random shuffle, simply change the `random_state` variable to another number to get a new shuffle state.

In [13]:
df_add = df_tracks[df_tracks['playlist_id'] == playlist_id].sample(frac = 1, random_state = 42)
df_add = df_add.reset_index(drop = True)
df_add['group'] = (df_add.index // 100) + 1
df_add.head()

,href,track.id,track.name,playlist_id,group
0,https://api.spotify.com/v1/playlists/730Ce3gbz...,5x5JM1BSB6vollcIzDocqT,The Climb,730Ce3gbzbasRtac5l8eXs,1
1,https://api.spotify.com/v1/playlists/730Ce3gbz...,05mAIVLkIWc2d1UBYZBCp8,1999,730Ce3gbzbasRtac5l8eXs,1
2,https://api.spotify.com/v1/playlists/730Ce3gbz...,5ojMDcYCghyRb4JSI7JL0Z,Starving,730Ce3gbzbasRtac5l8eXs,1
3,https://api.spotify.com/v1/playlists/730Ce3gbz...,1CuzzAbJ4q28V5JOLCzQVp,Starstrukk,730Ce3gbzbasRtac5l8eXs,1
4,https://api.spotify.com/v1/playlists/730Ce3gbz...,69RbpGQQzJopGwkiNvOxws,Papamericano,730Ce3gbzbasRtac5l8eXs,1


In [14]:
for i in df_add['group'].unique():
    df_tmp = df_add[df_add['group'] == i]
    tracks = ('spotify:track:' + df_tmp['track.id']).str.cat(sep=',')
    add_playlists_tracks(playlist_id, tracks, access_token)
    print('Done group ' + str(i))

Done group 1
Done group 2
Done group 3
Done group 4
Done group 5
Done group 6
Done group 7
Done group 8
Done group 9
Done group 10
Done group 11
